# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

### Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3256489 (TODO: check this!) artículos de Wikipedia repartidos en 3225 archivos. (NOTA: el README del corpus dice que son folders pero en realidad son files). Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

<br>
* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
           palabra (espacio) id

In [257]:
import pandas as pd
import numpy as np
import time

### Cargamos los datos

In [258]:
word_mapping = pd.read_csv('./Corpus_WiNER/document.vocab', sep=' ', header=None)
word_mapping.columns = ['word', 'id']

In [259]:
print(word_mapping.shape)
word_mapping.head()

(6931358, 2)


,word,id
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [260]:
word_mapping[word_mapping.word.isnull()]
# TODO: Decidir si quitar o no las filas con valores nulos, notar que esto puede
# provocar un error cuando se haga el mapping y esos ids dejen de estar presentes.
# word_mapping = word_mapping.dropna()

,word,id
17985,NaN,4004
38470,NaN,1272
49078,NaN,868
103471,NaN,262
117017,NaN,215
141695,NaN,158
184456,NaN,104


In [262]:
# TODO: Hay muchas palabras que tienen asignados los mismos id, tiene sentido removerlas?
print(word_mapping[word_mapping.id == 1].shape[0])
word_mapping[word_mapping.id == 1].head()

3819581


,word,id
3111777,龢,1
3111778,龟城叟集,1
3111779,龟兹语,1
3111780,龟仑语,1
3111781,龝,1


In [263]:
len(word_mapping['id'].unique())

16925

### Cargamos los artículos del Documento "0"

In [274]:
doc_0 = pd.read_csv('./Corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [275]:
doc_0.shape

(130209, 2)

In [276]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [277]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [278]:
doc_0 = doc_0.dropna()

In [279]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas

In [280]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: x.split(' '))

In [281]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con el primer artículo

In [282]:
article = doc_0[doc_0.art_ID == 431388]

Reconstruimos la primera oración del artículo

In [286]:
def sentence_decoder(sentence, word_mapping):
    new_sentence = []
    for w in sentence:
        mapped_w = word_mapping[word_mapping.id == np.int64(w)].word.iloc[0]
        new_sentence.append(mapped_w)

    return new_sentence 

In [287]:
sentence = article.loc[1, 'sentence']
sentence

['650',
 '5590',
 '753942',
 '12',
 '189',
 '243',
 '1',
 '2039',
 '47',
 '257',
 '682',
 '1',
 '164',
 '13',
 '1',
 '930',
 '84',
 '14',
 '650',
 '5590',
 '1',
 '8',
 '25',
 '102',
 '575',
 '8226',
 '11',
 '0',
 '3002',
 '1163',
 '11258',
 '2']

In [288]:
new_sentence = sentence_decoder(sentence, word_mapping)
new_sentence

IndexError: single positional indexer is out-of-bounds

### Notar que el error proviene de que algunos de los ids de las palabras se encuentran en word_mapping

In [290]:
# Ejemplos de ids que estan en la sentencia codificada pero no en word_mapping:
print(word_mapping[word_mapping.id == 0])
print(word_mapping[word_mapping.id == 753942])

Empty DataFrame
Columns: [word, id]
Index: []
Empty DataFrame
Columns: [word, id]
Index: []


In [76]:
# NO ESCALA!

# start = time.time()
# art_ID = 0
# for index, row in doc_0.iterrows():
#     if row['sentence'][0] == 'ID':
#         art_ID = row['sentence'][1]
#     else:
#         doc_0.loc[index, 'art_ID'] = art_ID

# end = time.time()
# print('Tiempo de demora: ', end - start)
# print('doc_0.shape: ', doc_0.shape)
# doc_0.head()

In [ ]:
# start = time.time()
# with open('./Corpus_WiNER/Documents/0') as file_0:
#     sentence = file_0.readline().strip().split(' ')
#     if sentence[0] == 'ID':
#         art_ID = sentence[1]
#         print('ID Articulo: ', art_ID)
#         df = pd.DataFrame(columns={'articleID', 'sentence'})
        
#         sentence = file_0.readline().strip().split(' ')
#         while sentence[0] != 'ID':
#             df = df.append({'articleID': art_ID, 'sentence': sentence}, ignore_index=True)
#             sentence = file_0.readline().strip().split(' ')
        
#     print('Termine de cargar el articulo')
#     end = time.time()
#     print('Tiempo de demora: ', end - start)
#     file_0.close()